In [2]:
import cv2

In [3]:
import numpy as np

In [4]:
import pyrealsense2 as rs

In [7]:
import time

In [19]:
# Configure streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
config.enable_stream(rs.stream.infrared,1280, 720, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)
dirname = 'D:\\20180626_test'
lastTime = 0
startTime = time.time()
c = 0
try:
    while True:

        # Wait for a coherent pair of frames: depth and color…infraredの処理適当
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        ir = frames.first(rs.stream.infrared)
        if not depth_frame or not color_frame:
            continue

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        ir_image = np.asanyarray(ir.get_data())
        
        nowTime = time.time()
        
        if nowTime - lastTime > 60 * 2:
            # csvファイルを保存する
            np.savetxt(dirname + '/depth_'+'{0:04d}'.format(c)+'.csv',depth_image,delimiter=',')
            #np.savetxt(dirname + '/color_b'+'{0:04d}'.format(c)+'.csv',color_image[:,:,0],delimiter=',')
            #np.savetxt(dirname + '/color_r'+'{0:04d}'.format(c)+'.csv',color_image[:,:,1],delimiter=',')
            #np.savetxt(dirname + '/color_g'+'{0:04d}'.format(c)+'.csv',color_image[:,:,2],delimiter=',')
            #print(ir_image.shape)
            #np.savetxt(dirname + '/ir_image_0'+'{0:04d}'.format(c)+'.csv',ir_image[:,:,0],delimiter=',')
            #np.savetxt(dirname + '/ir_image_1'+'{0:04d}'.format(c)+'.csv',ir_image[:,:,1],delimiter=',')
            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            #depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            #jpgファイルを保存する
            #cv2.imwrite(dirname + '/depth_'+'{0:04d}'.format(c)+'.jpg',depth_image)
            cv2.imwrite(dirname + '/color'+'{0:04d}'.format(c)+'.jpg',color_image)
            cv2.imwrite(dirname + '/ir_image'+'{0:04d}'.format(c)+'.jpg',ir_image)
            c += 1
            lastTime = time.time()
        if (nowTime-startTime) > 60 * 60 * 15 :
            break
#images = np.hstack((ir_image))
        
#        images = (ir_image)[:,::3]
#         print(images.shape)
#         images = cv2.bitwise_not(images)
        
#         plt.hist(images.ravel(),256,[0,256])
#         plt.show()
        
        # plt.imshow(images)
        # Show images
        #cv2.namedWindow('RealSense1', cv2.WINDOW_AUTOSIZE)
        #cv2.imshow('RealSense1', ir_image)
        #cv2.namedWindow('RealSense2', cv2.WINDOW_AUTOSIZE)
        #cv2.imshow('RealSense2', depth_colormap)
        #cv2.waitKey(1)

finally:

    # Stop streaming
    pipeline.stop()